In [25]:
cluster = {"kampfhandlungen": ["angriff", "luftangriff", "militaereinsatz", "töten", "tote", "zivilist", "schule", "krankenhaus", "offensive", "explosion", "is", "alqaida", "anschlaege", "militaeraktion", "aqap", "aqpa", "terrormiliz", "bus", "attentäter", "bombenanschläge", "bombardement", "bekennerschreiben", "tat", "angriff", "moschee", "gotteshaus", "flächenbrand", "entschlossen", "vorgehen", "streitkarft", "widerstand", "präsidentenpalast", "aufstandsbewegung"],
    "militaerische-unterstuetzung": ["kriegsteilnehmer", "kriegsparteien", "soldat", "bodentruppen", "transporter", "panzerabwehrraket", "waffenlieferungen", "gefechts", "soldat", "kairo", "aegypten", "bahrain", "jordanien", "senator", "senat", "kongress", "kammer", "veto", "pakistan", "islamabad", "nawaz", "sharif", "unterstützung"],
    "verhandlungen": ["verhandlungstisch", "feuerpause", "gesprächen", "gefangenen", "merkel", "austausch", "vertreter", "waffenruhe", "waffenstillstand", "kriegsteilnehmer", "konsultationen", "autonomie", "erklärung"],
    "humanitaer-klima": ["meteorologie", "wmo", "überschwemmungen", "regensaison", "sturm", "wirbelsturm", "zyklon", "super-zyklon"],
    "humanitaer-hunger": ["ernaehrungshilfe", "wfp", "unterernährung", "blockade", "hungern", "nahrungsmangel", "unwetter", "tragödie", "katastrophe", "arm", "terrorli", "blockade", "sanktion", "welthungerhilfe"],
    "humanitaer-krankheiten": ["pandemie", "epidemie", "cholera", "coronavirus", "corona", "who", "ausbruch", "anstecken", "hygiene"],
    "humanitaer-flucht": ["unhcr", "flüchtlingswerk", "flucht", "migration", "heimatländer", "somalia", "äthiopien", "schlepper", "belarus", "polen", "kenteren", "boot", "flüchtlingsboote", "ertrinken", "migrant", "iom", "flüchtling", "flüchtlingslager"],
    "humanitaer_Hilfe": ["geberkonferenz", "geld", "weltgemeinschaft", "zielgröße", "milliarde", "zusagen", "hilfe"],
    "wirtschaft": ["währungsverfall", "export", "qat", "konsum", "luxusgüter", "handel"]}

In [26]:
import pandas as pd
classified = pd.read_csv("classified_keywords_for_real.tsv", sep = "\t").to_dict("records") + pd.read_csv("classified_keywords.tsv", sep = "\t").to_dict("records") + pd.read_csv("classified_final_keywords_definitely.tsv", sep = "\t").to_dict("records")


for row in classified:
    if row["score"] == 1:
        cluster["kampfhandlungen"].append(row["keywords"])
    elif row["score"] == 2:
        cluster["verhandlungen"].append(row["keywords"])
    elif row["score"] == 3:
        cluster["humanitaer-klima"].append(row["keywords"])
    elif row["score"] == 4:
        cluster["humanitaer-hunger"].append(row["keywords"])
    elif row["score"] == 5:
        cluster["humanitaer-krankheiten"].append(row["keywords"])
    elif row["score"] == 6:
        cluster["humanitaer_Hilfe"].append(row["keywords"])
    elif row["score"] == 7:
        cluster["militaerische-unterstuetzung"].append(row["keywords"])
    elif row["score"] == 8:
        cluster["wirtschaft"].append(row["keywords"])
    elif row["score"] == 9:
        cluster["humanitaer-flucht"].append(row["keywords"])


In [27]:
import json

with open("../analysis/corpus.json", "r") as file: 
    corpus = json.load(file)

from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


In [28]:
for k, v in cluster.items():
    print(len(v))

208
58
80
23
65
40
20
23
18


In [29]:
cluster_based_on_similarity = {}
for topic, keywords in cluster.items():
    cluster_based_on_similarity[topic] = keywords.copy()
    for i, article in enumerate(corpus):
        for topic_kw in keywords:
            for article_kw in article["keywords"].keys():
                if article_kw not in cluster_based_on_similarity[topic]:
                    score = similar(topic_kw, article_kw)
                    if score > 0.8:
                        cluster_based_on_similarity[topic] += [article_kw]
                        

In [30]:
articles_with_topics = []

for article in corpus:
    article_kw = article["keywords"].keys()
    tmp_article_with_topic = {"id": article["id"], "title": article["title"]}
    for topic, topic_keywords in cluster_based_on_similarity.items(): 
        tmp_article_with_topic[topic] = len([kw for kw in article_kw if kw in topic_keywords])
    articles_with_topics.append(tmp_article_with_topic)    




In [31]:
import pandas as pd
cluster_per_article = pd.DataFrame(articles_with_topics)

classified = cluster_per_article[(cluster_per_article["militaerische-unterstuetzung"]>0)|(cluster_per_article["verhandlungen"]>0)|(cluster_per_article["kampfhandlungen"]>0)|(cluster_per_article["humanitaer-klima"]>0)|(cluster_per_article["humanitaer-hunger"]>0)|(cluster_per_article["humanitaer-krankheiten"]>0)|(cluster_per_article["humanitaer_Hilfe"]>0)|(cluster_per_article["humanitaer-flucht"]>0)|(cluster_per_article["wirtschaft"]>0)]
not_classified = cluster_per_article[(cluster_per_article["militaerische-unterstuetzung"]==0)&(cluster_per_article["verhandlungen"]==0)&(cluster_per_article["kampfhandlungen"]==0)&(cluster_per_article["humanitaer-klima"]==0)&(cluster_per_article["humanitaer-hunger"]==0)&(cluster_per_article["humanitaer-krankheiten"]==0)&(cluster_per_article["humanitaer_Hilfe"]==0)&(cluster_per_article["humanitaer-flucht"]==0)&(cluster_per_article["wirtschaft"]==0)]



In [33]:
classified.to_csv("articles_with_cluster.tsv", sep = "\t", index=False)

In [11]:
not_classified_corpus = []
flucht = []

for article in corpus:
    if article["id"] in not_classified["id"]:
        if "flucht" in article["title"].lower().replace("ü", "u"):
            flucht.append(article)
        else:
            not_classified_corpus.append(article)
        
        

sample = pd.DataFrame(not_classified_corpus).sample(20)
pd.concat([sample, pd.DataFrame(flucht)]).to_csv("articles_not_yet_classified.tsv", sep= "\t", index=False)

In [18]:
not_classified = []
for kw in pd.DataFrame(not_classified_corpus)["keywords"]:
    not_classified += list(kw.keys())

pd.DataFrame({"keywords": list(set(not_classified))}).to_csv("final_keywords_definitely.tsv", sep = "\t", index=False)



In [13]:
kw_not_classified = []
for article in corpus:
    if article["id"] in not_classified["id"]:
        kw_not_classified += list(article["keywords"].keys())

In [14]:
pd.DataFrame({"keywords": list(set(kw_not_classified))}).to_csv("keywords_to_classify_for_real.tsv", sep="\t", index=False)

In [15]:
for article in corpus:
    if article["id"] == 678:
        print(article)

{'title': 'Waffenruhe in Jemen', 'link': 'https://archiv.szarchiv.de/Portal/restricted/Start.act?articleId=A68925841', 'date': '19.10.2016', 'abstract': 'New York - Die Vereinten Nationen haben eine neue Waffenruhe im\nBürgerkrieg in Jemen angekündigt. Von Mittwoch, 22.59 Uhr Ortszeit, an\nwürden die Waffen schweigen, teilte der UN-Sonderbeauftragte für Jemen,\nIsmail Ould Sheikh Ahmed, mit. Alle Beteiligten hätten zugesagt, sich\ndaran zu halten. Die Ruhe gelte zunächst für 72 Stunden mit Option auf\n ...', 'author': '', 'surtitle': 'AUSLAND', 'source': 'SZ', 'id': 678, 'text': 'AUSLAND\nWaffenruhe in Jemen\nNew York – Die Vereinten Nationen haben eine neue Waffenruhe im Bürgerkrieg in Jemen angekündigt. Von Mittwoch, 22.59 Uhr Ortszeit, an würden die Waffen schweigen, teilte der UN-Sonderbeauftragte für Jemen, Ismail Ould Sheikh Ahmed, mit. Alle Beteiligten hätten zugesagt, sich daran zu halten. Die Ruhe gelte zunächst für 72 Stunden mit Option auf Verlängerung. In Jemen kämpfen schi

141